## Model

In [1]:
import transformers

print(transformers.__version__)

4.39.0


In [2]:
import pandas as pd
import numpy as np
import torch
import random
from torch.utils.data import Dataset, DataLoader
from transformers import set_seed, AutoTokenizer, AutoModel, DistilBertTokenizer, DistilBertForSequenceClassification

from sklearn.model_selection import train_test_split
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [3]:
import datasets
import random
from IPython.display import display, HTML

In [4]:
#!pip install datasets

In [5]:
torch.cuda.is_available()

True

## Helper functions

In [6]:
class AbstractsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: self.encodings[key][idx] for key in self.encodings}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)  # Note the dtype change for binary classification
        return item

    def __len__(self):
        return len(self.labels)

## Set seed

In [7]:
set_seed(6013)

## TinyBERT

In [8]:
dat = pd.read_csv('zotero_data.csv')
dat['abstract'] = dat['abstract'].astype(str)

In [9]:
labels = dat['fus_related'].tolist()
print("Label distribution:", np.bincount(labels))

Label distribution: [1794 1794]


In [10]:
tokenizer = AutoTokenizer.from_pretrained('huawei-noah/TinyBERT_General_4L_312D')
tokenized_data = tokenizer(list(dat['abstract']), padding=True, truncation=True, max_length=512, return_tensors="pt")

In [11]:
dataset = AbstractsDataset(tokenized_data, labels)

In [12]:
# Split indices into train and test sets first
train_idx, test_idx = train_test_split(list(range(len(dataset))), test_size=0.1, random_state=42)

# Split train indices into train and validation sets
train_idx, val_idx = train_test_split(train_idx, test_size=0.1, random_state=42)

# Create Subset datasets
train_dataset = torch.utils.data.Subset(dataset, train_idx)
val_dataset = torch.utils.data.Subset(dataset, val_idx)
test_dataset = torch.utils.data.Subset(dataset, test_idx)
print(f"Dataset size - train: {len(train_dataset)}, val: {len(val_dataset)}, test: {len(test_dataset)}")

Dataset size - train: 2906, val: 323, test: 359


In [13]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    learning_rate=0.001, 
    report_to=None,
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    logging_strategy="epoch",  # Log at the end of each epoch
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary', zero_division=1)  # Adjust zero_division here
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

## Hyper-param tuning

In [14]:
#! pip install ray[tune]

In [15]:
import ray
from ray import train, tune

In [16]:
ray.init()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Python version:,3.10.12
Ray version:,2.10.0


In [17]:
# Print the status of the Ray cluster.
print('''This cluster consists of
    {} nodes in total
    {} CPU resources in total
    {} Memory resources in total
    {} Object store memory resources in total
    '''.format(len(ray.nodes()), ray.cluster_resources()['CPU'], 
           ray.cluster_resources()['memory'] / (1024*1024*1024), 
           ray.cluster_resources()['object_store_memory'] / (1024*1024*1024)))

This cluster consists of
    1 nodes in total
    48.0 CPU resources in total
    153.51371326483786 Memory resources in total
    69.7829738613218 Object store memory resources in total
    


In [18]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained('huawei-noah/TinyBERT_General_4L_312D', num_labels=2)

In [19]:
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.create_optimizer_and_scheduler(num_training_steps=1000)
trainer.optimizer = torch.optim.AdamW(trainer.model.parameters(),lr=5e-5)
trainer.lr_scheduler = get_linear_schedule_with_warmup(trainer.optimizer, num_warmup_steps=500, num_training_steps=1000)

/home/re4mfy/.local/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:835: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tenso

In [20]:
def ray_hp_space(trial):
    return {
        "per_device_train_batch_size": tune.grid_search([8, 16, 32]),
        "learning_rate": tune.grid_search([2e-5, 3e-5, 5e-5]),
        "num_train_epochs": tune.grid_search([2, 3, 4])
    }

In [21]:
best_run = trainer.hyperparameter_search(n_trials=1, 
                                        direction="maximize",
                                        hp_space=ray_hp_space)

2024-04-10 01:06:13,179	INFO tune.py:613 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


== Status ==
Current time: 2024-04-10 01:06:13 (running for 00:00:00.30)
Using FIFO scheduling algorithm.
Logical resource usage: 0/48 CPUs, 0/4 GPUs (0.0/1.0 accelerator_type:RTX)
Result logdir: /tmp/ray/session_2024-04-10_01-05-40_645485_251574/artifacts/2024-04-10_01-06-13/_objective_2024-04-10_01-06-13/driver_artifacts
Number of trials: 27/27 (27 PENDING)
+------------------------+----------+-------+-----------------+--------------------+------------------------+
| Trial name             | status   | loc   |   learning_rate |   num_train_epochs |   per_device_train_bat |
|                        |          |       |                 |                    |                ch_size |
|------------------------+----------+-------+-----------------+--------------------+------------------------|
| _objective_0d645_00000 | PENDING  |       |           2e-05 |                  2 |                      8 |
| _objective_0d645_00001 | PENDING  |       |           3e-05 |                  2 |    

(_objective pid=254740) /home/re4mfy/.local/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
(_objective pid=254740) dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
(_objective pid=254740)   warnings.warn(
(_objective pid=254740) Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
(_objective pid=254740) /usr/local/lib/python3.10/dist-packages/torch/_utils.py:835: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage

== Status ==
Current time: 2024-04-10 01:07:24 (running for 00:01:10.88)
Using FIFO scheduling algorithm.
Logical resource usage: 4.0/48 CPUs, 4.0/4 GPUs (0.0/1.0 accelerator_type:RTX)
Result logdir: /tmp/ray/session_2024-04-10_01-05-40_645485_251574/artifacts/2024-04-10_01-06-13/_objective_2024-04-10_01-06-13/driver_artifacts
Number of trials: 27/27 (23 PENDING, 4 RUNNING)
+------------------------+----------+---------------------+-----------------+--------------------+------------------------+
| Trial name             | status   | loc                 |   learning_rate |   num_train_epochs |   per_device_train_bat |
|                        |          |                     |                 |                    |                ch_size |
|------------------------+----------+---------------------+-----------------+--------------------+------------------------|
| _objective_0d645_00000 | RUNNING  | 10.155.48.65:254740 |           2e-05 |                  2 |                      8 |
| _

  0%|          | 1/728 [00:04<49:41,  4.10s/it]
(_objective pid=254743) /home/re4mfy/.local/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead:  [repeated 3x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(_objective pid=254743) dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True) [repeated 3x across cluster]
(_objective pid=254743)   warnings.warn( [repeated 3x across cluster]
(_objective pid=254743) Detected kernel version 4.18.0, which is below the recommended 

...

(_objective pid=261209) {'loss': 0.3185, 'grad_norm': 5.679087162017822, 'learning_rate': 3.6400000000000004e-05, 'epoch': 4.0}
(_objective pid=261209) {'eval_loss': 0.3054695129394531, 'eval_accuracy': 0.8885448916408669, 'eval_f1': 0.8983050847457628, 'eval_precision': 0.8548387096774194, 'eval_recall': 0.9464285714285714, 'eval_runtime': 0.6574, 'eval_samples_per_second': 491.359, 'eval_steps_per_second': 31.946, 'epoch': 3.0} [repeated 2x across cluster]


  0%|          | 0/21 [00:00<?, ?it/s]


(_objective pid=261209) {'train_runtime': 56.5471, 'train_samples_per_second': 205.563, 'train_steps_per_second': 6.437, 'train_loss': 0.4686653535444658, 'epoch': 4.0}


100%|██████████| 364/364 [00:56<00:00,  9.74it/s]A
2024-04-10 01:24:49,838	INFO tune.py:1016 -- Wrote the latest version of all result files and experiment state to '/home/re4mfy/ray_results/_objective_2024-04-10_01-06-13' in 0.1619s.
2024-04-10 01:24:49,856	INFO tune.py:1048 -- Total run time: 1116.68 seconds (1116.37 seconds for the tuning loop).


== Status ==
Current time: 2024-04-10 01:24:49 (running for 00:18:36.53)
Using FIFO scheduling algorithm.
Logical resource usage: 1.0/48 CPUs, 1.0/4 GPUs (0.0/1.0 accelerator_type:RTX)
Result logdir: /tmp/ray/session_2024-04-10_01-05-40_645485_251574/artifacts/2024-04-10_01-06-13/_objective_2024-04-10_01-06-13/driver_artifacts
Number of trials: 27/27 (27 TERMINATED)
+------------------------+------------+---------------------+-----------------+--------------------+------------------------+-------------+
| Trial name             | status     | loc                 |   learning_rate |   num_train_epochs |   per_device_train_bat |   objective |
|                        |            |                     |                 |                    |                ch_size |             |
|------------------------+------------+---------------------+-----------------+--------------------+------------------------+-------------|
| _objective_0d645_00000 | TERMINATED | 10.155.48.65:254740 |          

In [22]:
for n, v in best_run.hyperparameters.items():
    print(n, v)

per_device_train_batch_size 16
learning_rate 5e-05
num_train_epochs 3


In [23]:
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.620300,0.449380,0.839009,0.845238,0.845238,0.845238
2,0.387600,0.436746,0.835913,0.826230,0.919708,0.750000
3,0.310700,0.192639,0.947368,0.950725,0.926554,0.976190


TrainOutput(global_step=546, training_loss=0.4395494845323947, metrics={'train_runtime': 33.3954, 'train_samples_per_second': 261.054, 'train_steps_per_second': 16.35, 'total_flos': 125007369080832.0, 'train_loss': 0.4395494845323947, 'epoch': 3.0})

In [24]:
trainer.evaluate(eval_dataset=test_dataset)

{'eval_loss': 0.3075887858867645,
 'eval_accuracy': 0.883008356545961,
 'eval_f1': 0.8894736842105263,
 'eval_precision': 0.8125,
 'eval_recall': 0.9825581395348837,
 'eval_runtime': 0.4507,
 'eval_samples_per_second': 796.471,
 'eval_steps_per_second': 51.027,
 'epoch': 3.0}